In [1]:
#reading packages
import os
import pandas as pd
import rsa
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP, AES
from Crypto.Util.Padding import pad, unpad

# Data Set

In [2]:
pd.set_option('display.max_columns', None)
df_encrypt = pd.read_csv('dataset_febrl3.csv',low_memory=False)
df_encrypt.head()

,rec_id,given_name,surname,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
0,rec-1496-org,mitchell,green,wallaby place,delmar,cleveland,2119,sa,19560409.0,1804974
1,rec-552-dup-3,harley,mccarthy,pridhamstreet,milton,marsden,3165,nsw,19080419.0,6089216
2,rec-988-dup-1,madeline,mason,hoseason street,lakefront retrmnt vlge,granville,4881,nsw,19081128.0,2185997
3,rec-1716-dup-1,isabelle,NaN,gundulu place,currin ga,utakarra,2193,wa,19921119.0,4314184
4,rec-1213-org,taylor,hathaway,yuranigh court,brentwood vlge,NaN,4220,nsw,19991207.0,9144092


In [4]:
df_encrypt.shape

(5000, 10)

# Generation of keys

In [4]:
key = RSA.generate(2048) # Generating a new RSA key pair
with open('public_key.pem', 'wb') as file:
    file.write(key.publickey().export_key()) # Saving the public and private keys
with open('private_key.pem', 'wb') as file:
    file.write(key.export_key())

# Opening a dataset for encryption

In [5]:
with open('df_encrypt.csv', 'rb') as file:   # Opening the dataset file for encryption in binary format
    data = file.read()

# Encryption Key Generation

In [6]:
symmetric_key = os.urandom(32) # Generating a random 256-bit encryption key
iv = os.urandom(16) # Initializing the AES cipher in CBC mode with a random IV
cipher = AES.new(symmetric_key, AES.MODE_CBC, iv=iv)
padded_data = pad(data, AES.block_size) # Padding the data to a multiple of 16 bytes 

# Encryption

In [7]:
encrypted_data = cipher.encrypt(padded_data) # Encrypting the data with AES
rsa_cipher = PKCS1_OAEP.new(key.publickey()) # Encrypting the symmetric key with RSA
encrypted_symmetric_key = rsa_cipher.encrypt(symmetric_key)

with open('encrypted_data.bin', 'wb') as file:
    file.write(encrypted_symmetric_key + iv + encrypted_data) # Saving the encrypted data and symmetric key to a new file

with open('encrypted_data.bin', 'rb') as file: # Opening the encrypted data file for decryption in binary mode
    encrypted_data = file.read()

encrypted_symmetric_key = encrypted_data[:key.size_in_bytes()]  # Extracting the encrypted symmetric key and IV from the file
iv = encrypted_data[key.size_in_bytes():key.size_in_bytes()+AES.block_size]

In [13]:
with open('encrypted_data.csv', 'wb') as file: # Saving a csv of encrypted data
    file.write(encrypted_data)

# Decryption

In [13]:
rsa_cipher = PKCS1_OAEP.new(key)          # Decrypting the symmetric key with RSA
symmetric_key = rsa_cipher.decrypt(encrypted_symmetric_key) 

cipher = AES.new(symmetric_key, AES.MODE_CBC, iv=iv) # Initializing the AES cipher with the decrypted symmetric key and IV

decrypted_data = unpad(cipher.decrypt(encrypted_data[key.size_in_bytes()+AES.block_size:]), AES.block_size) # Decrypting the data with AES

with open('decrypted_data.csv', 'wb') as file: # Saving a csv of the decrypted data
    file.write(decrypted_data)